<a href="https://colab.research.google.com/github/webjjang/data_anal/blob/main/ch03_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 필요한 파일 다운 받기
import gdown
gdown.download("https://bit.ly/3RhoNho", "ns_202104.csv", quiet=False)


Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:00<00:00, 112MB/s]


'ns_202104.csv'

In [3]:
# pandas로 데이터 일어오기
import pandas as pd
ns_df = pd.read_csv("ns_202104.csv", low_memory=False)
# 13 의미없는 열 제거
ns_book = ns_df.loc[:, "번호":"등록일자"]

In [7]:
selected_rows = ns_df["출판사"] == "한빛미디어"
ns_book2 = ns_book[selected_rows]
count_df = ns_book[["도서명", "저자", "ISBN", "권", "대출건수"]]
group_df = count_df.groupby(by=["도서명", "저자", "ISBN", "권"], dropna=False)
loan_count = group_df.sum()
# 도서 대여가 중복이 되는 데이터
# 처음 데이터는 중복이 되지 않았다. - False, 2번 데이터부터는 중복이 되었다. - True
dup_rows = ns_book.duplicated(subset=["도서명", "저자", "ISBN", "권"])
# 중복이 안되는 데이터
unique_rows = ~dup_rows
ns_book3 = ns_book[unique_rows].copy()
ns_book3.set_index(["도서명", "저자", "ISBN", "권"], inplace=True)
ns_book3.update(loan_count)
ns_book4 = ns_book3.reset_index()
ns_book4.head()

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈 지음,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형 지음,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,9788934990833,NaN,5,김영사,2021,NaN,NaN,NaN,1,0,2021-03-19
